In [1]:
import pandas as pd
import os

# --- Config ---
excel_path = r"E:\STOCKS\Portfolio.xlsx"  # <-- Update this
sheet_name = "Dashboard"
col_ticker, col_weight = "B", "L"
row_start, row_end = 7, 36  # inclusive
output_file = "../data/portfolio_weights.csv"
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# === LOAD TICKERS AND WEIGHTS ===
nrows = row_end - row_start + 1
df = pd.read_excel(
    excel_path,
    sheet_name=sheet_name,
    usecols=f"{col_ticker},{col_weight}",
    skiprows=row_start - 1,
    nrows=nrows,
    header=None,
    names=["Ticker", "Weight"]
).dropna()

# === CLEANING ===
df["Ticker"] = df["Ticker"].astype(str).str.strip().replace({"$": "SGOV"})
df["Weight"] = pd.to_numeric(df["Weight"], errors="coerce")

# === GROUP AND SUM DUPLICATE TICKERS ===
df = df.groupby("Ticker", as_index=False)["Weight"].sum()

# === BUILD FINAL WEIGHTS DICTIONARY ===
WEIGHTS = dict(zip(df["Ticker"], df["Weight"]))
total = sum(WEIGHTS.values())

# === DEBUG PRINT (Sorted Descending) ===
print("✅ Parsed portfolio weights:\n")
for ticker, weight in sorted(WEIGHTS.items(), key=lambda x: x[1], reverse=True):
    print(f'{ticker} = {weight * 100:.2f}%')
print(f"\n✅ Total portfolio weight: {total * 100:.2f}%")

# === VALIDATION ===
if not abs(total - 1.0) < 1e-6:
    raise ValueError(f"❌ Portfolio weights sum to {total:.4f}, expected 1.0.")
print("✅ Portfolio weights loaded and validated successfully.")

# === EXPORT TO CSV ===

# Sort: non-zero weights (descending), then zero weights (alphabetical)
nonzero_df = df[df["Weight"] > 0].sort_values(by="Weight", ascending=False)
zero_df = df[df["Weight"] == 0].sort_values(by="Ticker")
export_df = pd.concat([nonzero_df, zero_df], ignore_index=True)

# Write to CSV
os.makedirs(os.path.dirname(output_file), exist_ok=True)
export_df.to_csv(output_file, index=False)
print(f"✅ Wrote: {output_file}")


✅ Parsed portfolio weights:

SGOV = 38.38%
ULTY = 15.87%
RDDT = 14.69%
GOOGL = 9.79%
META = 9.05%
ASTS = 7.34%
QQQM = 4.87%
BRK/B = 0.00%
HOOD = 0.00%
INTU = 0.00%
NBIS = 0.00%
SHOP = 0.00%
SNOW = 0.00%
TQQQ = 0.00%

✅ Total portfolio weight: 100.00%
✅ Portfolio weights loaded and validated successfully.
✅ Wrote: ../data/portfolio_weights.csv
